In [41]:
from sdm.utils import set_project_wd
set_project_wd()

Current Working Directory: /Users/matthewwhittle/Data Science/shefflied-bats


In [42]:
import geopandas as gpd
import pandas as pd
from leafmap.foliumap import Map

In [43]:
bats = gpd.read_parquet('data/processed/sybg-bats.parquet')
bats = bats[bats.accuracy <= 100]
bats.drop(columns=['grid_square_geom'], inplace=True)
bats.head()

,grid_reference,species_raw,activity_type,source_data,date,latin_name,common_name,genus,x,y,accuracy,geometry
0,NZ115084,Common pipistrelle,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1274054400000,""Gr...",2010-05-17 00:00:00,Pipistrellus pipistrellus,Common Pipistrelle,Pipistrellus,411550.0,508450.0,100.0,POINT (411550.000 508450.000)
1,NZ14640021,Pipistrellus sp.,Unknown,"{""Recorder"":""Giles Manners"",""Date"":14043456000...",2014-07-03 00:00:00,Pipistrellus sp.,Unidentified Pipistrelle,Pipistrellus,414645.0,500215.0,10.0,POINT (414645.000 500215.000)
2,NZ20291106,Soprano pipistrelle,Unknown,"{""Recorder"":""Natural England Volunteer Bat Roo...",2013-11-28 00:00:00,Pipistrellus pygmaeus,Soprano Pipistrelle,Pipistrellus,420295.0,511065.0,10.0,POINT (420295.000 511065.000)
3,NZ170014,Unidentified bat species,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1283212800000,""Gr...",2010-08-31 00:00:00,Unknown,Unidentified Bat,Unknown,417050.0,501450.0,100.0,POINT (417050.000 501450.000)
4,NZ185116,Unidentified bat species,Roost,"{""Recorder"":""BCT\/NE"",""Date"":1242777600000,""Gr...",2009-05-20 00:00:00,Unknown,Unidentified Bat,Unknown,418550.0,511650.0,100.0,POINT (418550.000 511650.000)


In [44]:
bats_subset = bats[
    (bats.latin_name == 'Myotis mystacinus') & (bats.activity_type == 'Foraging')
]

In [45]:
import json
source_data = pd.DataFrame(bats_subset.source_data.apply(json.loads).tolist())
bats_subset.drop(columns=['source_data'], inplace=True)
bats_subset.reset_index(drop=True, inplace=True)
# Add this as additional columns
bats_subset = bats_subset.join(source_data)

In [46]:
bats_subset = bats_subset.to_crs(epsg=4326)
m = Map()
m.add_basemap('OpenStreetMap')
m.add_basemap("HYBRID")
m.add_gdf(bats_subset, layer_name='Bats', )
# center the map
m.zoom_to_gdf(bats_subset)
m